HW 3 of Computational methods 540

Review of basic statistics:MVN

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib as mpl
import scipy.stats as st
from scipy.stats import *

In [32]:
# x = np.linspace(0, 5, 10)
# mean = np.array([5,1,2])
# cov = np.array([[20,11,-2], [11,2,8],[-2,8,7]])
# mvn3 = multivariate_normal(x,mean,cov)

In [33]:

d = 3
mean = np.matrix([5,1,2])
cov = np.matrix([[20,11,-2], [11,2,8],[-2,8,7]])
eval = np.linalg.eigvals(cov)
L = linalg.cholesky(cov)
n = 50  # Samples to draw
X = np.random.normal(size=(d, n))
# Apply the transformation
Y = L.dot(X) + mean

LinAlgError: 2-th leading minor of the array is not positive definite